In [ ]:

#name of bands
inBands = ee.List(['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B11','B12']);
outBands = ee.List(['Aerosol','blue','green','red','re1','re2','re3','nir','re4','waterVapor','swir1','swir2']);
 
CloudCoverMax = 20
valid_percentage_value = 1
#function to get the data
def importData(studyArea,startDate,endDate):
 
    # Get Sentinel-2 data
    s2s =(ee.ImageCollection('COPERNICUS/S2_SR')
          .filterDate(startDate,endDate)
          .filterBounds(studyArea)
          .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',CloudCoverMax))
          .filter(ee.Filter.lt('CLOUD_COVERAGE_ASSESSMENT',CloudCoverMax)))
    
    #sentinel bands are in scale of 0.0001
    def scaleBands(img):
        prop = img.toDictionary()
        t = (img.select(['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B11','B12'])
             .divide(10000))
        t = (t.addBands(img.select(['QA60'])).set(prop)
            .copyProperties(img,['system:time_start','system:footprint']))

        return ee.Image(t)

    def cloudpercentage_overregion(img):
        qa = img.select('QA60').clip(studyArea)
        cloudBitMask = 1024;
        cirrusBitMask = 2056;
        cloudmask = qa.bitwiseAnd(cloudBitMask).eq(0).bitwiseAnd(qa.bitwiseAnd(cirrusBitMask).eq(0));

        nocloudpixels = img.select('B9').clip(studyArea).updateMask(cloudmask);

        numofpixels = img.select('B9').clip(studyArea).reduceRegion(
                  reducer= ee.Reducer.count(), 
                  geometry= studyArea, 
                  scale= 60,
        );

        validpixels = nocloudpixels.reduceRegion(
              reducer= ee.Reducer.count(), 
              geometry= studyArea, 
              scale= 60,
              );

        valid_percentage = ee.Number(validpixels.get('B9')).divide(ee.Number(numofpixels.get('B9')))

        return img.set('numofpixels', ee.Number(numofpixels.get('B9'))).set('validpixels', ee.Number(validpixels.get('B9'))).set('valid_percentage', ee.Number(valid_percentage))

    def getimagedates(img):
        return ee.Feature(None, {'date': img.date().format('YYYY-MM-dd')}) 
    
    
    s2s = s2s.map(cloudpercentage_overregion).filter(ee.Filter.eq('valid_percentage',valid_percentage_value))
    valid_percentage = s2s.reduceColumns(ee.Reducer.toList(), ["valid_percentage"]).get('list')
    dates = s2s.map(getimagedates).distinct('date').aggregate_array('date')
    s2s = s2s.map(scaleBands)
    s2s = s2s.select(inBands,outBands)
    
    return s2s, valid_percentage, dates

In [ ]:
def clp(img) :
  return img.clip(studyArea).toFloat()

In [ ]:
year1,month1,day1= [ int(x) for x in input('enter your first date').split(',')]
year2,month2,day2= [ int(x) for x in input('enter your second date').split(',')]

startDate = ee.Date.fromYMD(year1,month1,day1)
endDate = ee.Date.fromYMD(year2,month2,day2)
studyArea = roi
#gee assets to get the study area
print("Getting images") 
img, valid_percentage, dates = importData(studyArea, startDate,endDate)

In [ ]:
dates.getInfo()

In [ ]:
imgclipped = img.map(clp)

In [ ]:
count = img.size()
range = img.reduceColumns(ee.Reducer.minMax(), ["system:time_start"])
print('Date range: ', ee.Date(range.get('min')), ee.Date(range.get('max')))

In [ ]:
geemap.ee_export_image_collection(imgclipped,scale = 10, out_dir="/content/drive/Shareddrives/My Drive2/colab_agriculture",region = studyArea)